In [31]:
from Bio.Seq import Seq

cwd = 'mito-annotation-validation\\'

## input files
sample = '4A'

#fasta_file = cwd + 'mito-genomes-reformatted\\' + sample + '_mito-nodes.fasta'
fasta_file = cwd + sample + '.fasta'
tbl_file = cwd  + sample + '.tbl'

# fasta_file = cwd + 'final_submission_files\\' + sample + '.fsa'
# tbl_file = cwd + 'final_submission_files\\' + sample + '.tbl'


start_codons = ['ATA', 'ATG']
stop_codons = ['TAA', 'TAG']


def reorientate(sequence, start_num):
    seq_end = sequence[:start_num]
    seq_start = sequence[start_num:]
    new_sequence = seq_start + seq_end
    return new_sequence

def parse_fasta(fasta_file): # expects a single fasta per entry
    header = ''
    sequence = ''
    for line in open(fasta_file):
        line = line.rstrip()
        if line[0] == '>':
            header = line
        else:
            sequence += line
    print (header, len(sequence))
    return header, sequence

def parse_tbl(tbl_file, sequence, start_num = 0,):
    with open(sample + '.tbl', 'w') as tbl_handle:
        data_lookup = {}
        cur_set = []

        all_data = []

        for line in open(tbl_file):
            if line.startswith('>'):
                header = line.rstrip()
                tbl_handle.writelines(header + '\n')
            else:
                elements = line.rstrip().split('\t')
                if elements[0] != '':
                    if start_num != 0: # re-adjust gene coordinates
                        for i in [0,1]:
                            if int(elements[i]) < start_num:
                                genome_difference = len(sequence) - start_num
                                new_dig = genome_difference + int(elements[i])

                            else:
                                new_dig = int(elements[i]) - start_num
                            elements[i] = str(new_dig)
                    # fix rRNA gene line        
                    if elements[2] == 'rRNA':
                        cur_set.extend(['gene','rRNA'])
                        all_data.append(cur_set)
                        cur_set = []
                    cur_set.extend(elements)

                else:
                    cur_set.extend(elements[-2:])

                    all_data.append(cur_set)
                    cur_set = []    
        sorted_data = sorted(all_data, key=lambda x: int(x[0]))
        for cur_data in sorted_data:
            data_lookup.setdefault(cur_data[2], [])
            data_lookup[cur_data[2]].append(cur_data)
            feature_start = cur_data[0]
            feature_end = cur_data[1]
            if not cur_data[2] == 'D-loop':
                tbl_handle.writelines('\t'.join(cur_data[:3]) + '\n\t\t\t' + '\t'.join(cur_data[3:]) + '\n')
            
            rev_comp = False
            if int(feature_start) > int(feature_end):
                rev_comp = True
                feature_start, feature_end = feature_end, feature_start
                
            print (cur_data)
            
            if cur_data[2] == 'rRNA':
                cur_seq = sequence[int(feature_start)-1:int(feature_end)]
                my_seq = Seq(cur_seq)
                print (my_seq)
            
            if cur_data[2] == 'CDS':
                cur_seq = sequence[int(feature_start)-1:int(feature_end)]
                
                
                # translate the data to amino acid
                my_seq = Seq(cur_seq)
                if rev_comp:
                    my_seq = my_seq.reverse_complement()
                s_codon = my_seq[:3]
                trans = my_seq.translate(table=5)
                stop_count = trans.count('*')

                if len(cur_seq) != len(trans)*3 or stop_count != 1 or s_codon not in start_codons or not trans.endswith('*'):
                    print ('\n\t\t'+cur_data[4])
                    print ('\t\t\tFIX THIS', len(cur_seq), len(trans)*3,len(cur_seq)%3,  stop_count)
                    print (my_seq)
                    print ('---------------------\n' + trans + '\n---------------------coordinates', feature_start, feature_end, int(feature_end)-int(feature_start))

########   
                    t_start = int(feature_start) - 1 -90
                    t_stop =  int(feature_end) +  120 - 24

########                    
                    test = Seq(sequence[t_start:t_stop])
                    if rev_comp:
                        test = test.reverse_complement()
                    test_trans = test.translate(table=5)
                    print (test, len(test), len(test_trans) * 3 )
                    print ('\n----------------coordinates', t_start, t_stop, t_stop-t_start)
                    print (test_trans)
                    print('\n\n\n')


        return data_lookup

header, sequence = parse_fasta(fasta_file)
sequence = reorientate(sequence, 0)

with open( sample + '.fsa', 'w') as f_handle:
    f_handle.writelines(header + '\n' + sequence)
tbl_lookup = parse_tbl(tbl_file, sequence, 0)

for d,v in tbl_lookup.items():
    print (d, len(v))
    for vv in v:
        print (vv)

>Sample_4A  [organism=Aphelenchoides besseyi] [location=mitochondrion] [moltype=genomic DNA] [gcode=5] 13114
['20', '455', 'D-loop', 'note', 'control region']
['611', '2041', 'gene', 'gene', 'cox1']
['611', '2041', 'CDS', 'product', 'cytochrome c oxidase subunit 1']

		cytochrome c oxidase subunit 1
			FIX THIS 1431 1431 0 0
TGATTTGAAAGTTCTAATCATAAAGATATCGGAATGCTTTATTTTTTATTTGGGTTTTGATCTGGTATATTGGGTACAAGGTTGTCTATAATTATTCGTTTTGAGTTGGCTAAGCCCGGCTTATTGTTTGAAAATGGGCAACTTTATAATAGAATCATTACTGCTCATGCGTTATTAATGATTTTTTTTATAGTTATGCCTAGAATGGTAGGAGGTTTTGGTAATTGGCTTTTACCTTTAATATTGGGATCTCCAGATATAAGGTTTCCACGTTTAAATAATTTGAGTTTTTGATTGTTACCTGTTTCTATGTTTTTGTTGTTAGATTCGAGTTTAGTTGATATGGGGGCTGGGACTAGATGGGTTGTGTACCCACCTTTAAGAACCTTAGGACATCCCGGTTACAGAGTTGATTTAGCAATCTTTAGTCTACATTGCGCTGGGGTTAGTTCTATTTTAGGCGGTATTAATTTTATAAGAACCACAAAGAATTTACGAAGTAGATCAATTTCTTTAGAACATATAAGATTATTTGTTTGAACTATTTTTGTTACTGTTTTTTTGTTAGTTTTATCTTTACCTGTTTTGGCGGGAGCTATCACTATATTATTGACTGATCGAAATTTAAATACTTCTTTTTTTGACCCTAGAATGGGGGGTAACCCTTTAATTT